In [1]:
#importing the libraries needed for ML
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

KeyboardInterrupt: 

In [ ]:
data = pd.read_csv(r'..\Datasets\Parkinsons Train Data.csv', header = 0)
data.head()

In [ ]:
#setting the dependent variable
y = data.iloc[:, -1].values
y

In [ ]:
X = data.iloc[:, :-1].values
X

In [ ]:
X.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
standard_X = StandardScaler()
standard_scaled_X = standard_X.fit_transform(X)
standard_scaled_X

In [ ]:
def standardScale(X):
    X = standard_X.transform(X)
    return X

In [ ]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
minmax_scaled_X = min_max_scaler.fit_transform(X)
minmax_scaled_X

In [ ]:
def minmaxScale(X):
    X = min_max_scaler.transform(X)
    return X

In [ ]:
data.columns

In [ ]:
fixed_cols = ['AC', 'NDH', 'HTM', 'Minimum pitch', 'Maximum pitch', 'Number of pulses', 'Number of periods',
              'Mean period', 'Fraction of locally unvoiced frames', 'Number of voice breaks', 'Degree of voice breaks']
jitter_cols = ['Jitter(local)', 'Jitter(local, absolute)', 'Jitter (rap)','Jitter (ppq5)', 'Jitter (ddp)']
shimmer_cols = ['Shimmer (local)', 'Shimmer (local, db)', 'Shimmer (apq3)', 'Shimmer (apq5)', 'Shimmer (apq11)',
                'Shimmer (dda)']
pitch_cols = ['Median Pitch', 'Mean Pitch']

In [ ]:
def getBestNeighbors(X_train, X_test, y_train, y_test):
    from sklearn.neighbors import KNeighborsClassifier
    training_accuracy = []
    test_accuracy = []
    # try n_neighbors from 1 to 100
    neighbors_settings = range(1, 100)
    index = 0
    max_accuracy = 0
    
    for n_neighbors in neighbors_settings:
        # build the model
        knn = KNeighborsClassifier(n_neighbors=n_neighbors)
        knn.fit(X_train, y_train)
        # record training set accuracy
        training_accuracy.append(knn.score(X_train, y_train))
        # record test set accuracy
        accuracy = knn.score(X_test, y_test)
        test_accuracy.append(accuracy)
        if(accuracy > max_accuracy):
            max_accuracy = accuracy
            index = n_neighbors
        
    print("Best Accuracy:", max_accuracy)
    print("N_neighbors: ", index)
    return index

In [ ]:
def getKNN(X, y):
    
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, random_state=12, test_size = 0.2)
    # Train
    from sklearn.neighbors import KNeighborsClassifier
    bestNoOfNeighbors = getBestNeighbors(X_train, X_test, Y_train, Y_test)
    model = KNeighborsClassifier(n_neighbors=bestNoOfNeighbors)
    model.fit(X_train, Y_train)

    # Evaluate
    print("Accuracy for training set -> ", end = "")
    Y_hat = [round(yhat) for yhat in model.predict(X)]
    print(accuracy_score(y, Y_hat)) # Full set accuracy
    Y_hat = [round(yhat) for yhat in model.predict(X_test)]
    print("Accuracy for test set -> ", end = "")
    print(accuracy_score(Y_test, Y_hat)) # Test set accuracy

    #Validations
    y_pred = model.predict(X_test)

    #Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(Y_test, y_pred) # Calulate Confusion matrix for test set.
    print(cm)

    from sklearn.model_selection import cross_val_score
    accuracies = cross_val_score(estimator = model, X = X_train, y = Y_train, cv = 10)
    print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
    print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

In [ ]:
for jitter in jitter_cols:
    for shimmer in shimmer_cols:
        for pitch in pitch_cols:
            
            fixed_cols.append(jitter)
            fixed_cols.append(shimmer)
            fixed_cols.append(pitch)
            print(fixed_cols)
            print("\n")
            print("Added columns to the dataset are:", jitter, shimmer, pitch)
            print("\n")
            X1 = data[fixed_cols].values
            print("---------Standard Scaled---------")
            print("\n")
            standard_X = StandardScaler()
            standard_scale_X = standard_X.fit_transform(X1)
            standard_scale_X
            getKNN(standard_scale_X, y)
            print("\n")
            print("---------MinMax Scaled---------")
            print("\n")
            min_max_scaler = preprocessing.MinMaxScaler()
            minmax_scale_X = min_max_scaler.fit_transform(X1)
            minmax_scale_X
            getKNN(minmax_scale_X, y)
            fixed_cols.remove(jitter)
            fixed_cols.remove(shimmer)
            fixed_cols.remove(pitch)
            print("\n")

In [ ]:
X1

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
minmax_scale_X = min_max_scaler.fit_transform(X1)
minmax_scale_X

In [ ]:
fixed_cols

In [ ]:
getKNN(standard_scaled_X, y)

In [ ]:
getKNN(minmax_scaled_X, y)

In [ ]:
minmax_scaled_X

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train, X_test, Y_train, Y_test = train_test_split(minmax_scaled_X, y, random_state=12, test_size = 0.2)
# Train
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=10)
print(model)
model.fit(X_train, Y_train)

# Evaluate
print("Accuracy for training set -> ", end = "")
Y_hat = [round(yhat) for yhat in model.predict(minmax_scaled_X)]
print(accuracy_score(y, Y_hat)) # Full set accuracy
Y_hat = [round(yhat) for yhat in model.predict(X_test)]
print("Accuracy for test set -> ", end = "")
print(accuracy_score(Y_test, Y_hat)) # Test set accuracy

#Validations
y_pred = model.predict(X_test)

#Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, y_pred) # Calulate Confusion matrix for test set.
print(cm)

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = model, X = X_train, y = Y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

In [ ]:
standard_scaled_X.shape

In [ ]:
import joblib
joblib.dump(model, 'finalModel_Deployment.pkl')
print("Model dumped!")

In [ ]:
model_columns = list(data.columns)
model_columns.remove('status')
model_columns

In [ ]:
joblib.dump(model_columns, 'model_columns_final.pkl')
print("Models columns dumped!")

In [ ]:
data.columns

In [ ]:
minmax_scaled_X.shape